In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import DenseNet201
# from keras.applications.densenet import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 35534421
EPOCHS = 50
size = 64
batchsize = 256
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize
NCATS = 340

In [3]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [4]:
K.clear_session()

model = DenseNet201(input_shape=(size, size, 1), weights=None, classes=NCATS)

sgd = SGD(0.0005, momentum=0.9, nesterov=True)
sgd = NormalizedOptimizer(sgd, normalization='l2')

model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   3136        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
valid_df = pd.read_csv('../input/valid_balance.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, 
                               preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [7]:
train_datagen = image_generator_xd(df_path = '../input/train_all_balance.csv',
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [8]:
model_prefix = 'densenet201_{}_lw{}_sgd'.format(size, lw)
fold = 9
snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                   nb_snapshots=1,
                                   size = size,
                                   init_lr=1e-3, 
                                   fold=fold)

callbks = snapshot.get_callbacks(model_prefix = model_prefix)

# callbks = [
#     ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
#                       min_delta=0.005, mode='max', cooldown=3, verbose=1),
#     callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
#                                 monitor='val_categorical_accuracy', 
#                                 mode = 'max', save_best_only=True, verbose=1),
#     callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
# ]

In [9]:
model.load_weights('models/densenet201_raw_head_64_lw6.model')

In [10]:
hists = []
hist = model.fit_generator(  
    train_datagen, steps_per_epoch=STEPS, ep ochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Stochastic weight averaging selected for last 15 epochs.
Epoch 1/50
2777/2776 [==============================] - 898s 323ms/step - loss: 0.7560 - categorical_crossentropy: 0.7560 - categorical_accuracy: 0.8046 - top_3_accuracy: 0.9258 - val_loss: 0.7332 - val_categorical_crossentropy: 0.7332 - val_categorical_accuracy: 0.8069 - val_top_3_accuracy: 0.9301

Epoch 00001: val_categorical_accuracy improved from -inf to 0.80694, saving model to ./models/densenet201_64_lw6_sgd.model
Epoch 2/50
2777/2776 [==============================] - 867s 312ms/step - loss: 0.7556 - categorical_crossentropy: 0.7556 - categorical_accuracy: 0.8042 - top_3_accuracy: 0.9262 - val_loss: 0.7329 - val_categorical_crossentropy: 0.7329 - val_categorical_accuracy: 0.8073 - val_top_3_accuracy: 0.9309

Epoch 00002: val_categorical_accuracy improved from 0.80694 to 0.80732, saving model to ./models/densenet201_64_lw6_sgd.model
Epoch 3/50
2777/2776 [==============================] - 866s 312ms/step - loss: 0.7538 - cat

2777/2776 [==============================] - 867s 312ms/step - loss: 0.7548 - categorical_crossentropy: 0.7548 - categorical_accuracy: 0.8047 - top_3_accuracy: 0.9258 - val_loss: 0.7226 - val_categorical_crossentropy: 0.7226 - val_categorical_accuracy: 0.8093 - val_top_3_accuracy: 0.9322

Epoch 00022: val_categorical_accuracy did not improve from 0.80965
Epoch 23/50
2777/2776 [==============================] - 868s 312ms/step - loss: 0.7541 - categorical_crossentropy: 0.7541 - categorical_accuracy: 0.8054 - top_3_accuracy: 0.9262 - val_loss: 0.7225 - val_categorical_crossentropy: 0.7225 - val_categorical_accuracy: 0.8093 - val_top_3_accuracy: 0.9321

Epoch 00023: val_categorical_accuracy did not improve from 0.80965
Epoch 24/50
2777/2776 [==============================] - 867s 312ms/step - loss: 0.7542 - categorical_crossentropy: 0.7542 - categorical_accuracy: 0.8050 - top_3_accuracy: 0.9259 - val_loss: 0.7219 - val_categorical_crossentropy: 0.7219 - val_categorical_accuracy: 0.8093 - 


Epoch 00043: val_categorical_accuracy did not improve from 0.81150
Epoch 44/50
2777/2776 [==============================] - 871s 314ms/step - loss: 0.7420 - categorical_crossentropy: 0.7420 - categorical_accuracy: 0.8069 - top_3_accuracy: 0.9278 - val_loss: 0.7145 - val_categorical_crossentropy: 0.7145 - val_categorical_accuracy: 0.8101 - val_top_3_accuracy: 0.9326

Epoch 00044: val_categorical_accuracy did not improve from 0.81150
Epoch 45/50
2777/2776 [==============================] - 870s 313ms/step - loss: 0.7441 - categorical_crossentropy: 0.7441 - categorical_accuracy: 0.8074 - top_3_accuracy: 0.9274 - val_loss: 0.7157 - val_categorical_crossentropy: 0.7157 - val_categorical_accuracy: 0.8114 - val_top_3_accuracy: 0.9326

Epoch 00045: val_categorical_accuracy did not improve from 0.81150
Epoch 46/50
2777/2776 [==============================] - 871s 314ms/step - loss: 0.7463 - categorical_crossentropy: 0.7463 - categorical_accuracy: 0.8068 - top_3_accuracy: 0.9270 - val_loss: 0.7

KeyboardInterrupt: 

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
model.load_weights('./models/{}_swa.model'.format(model_prefix))
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]

import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}_swa.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

In [ ]:
model.load_weights('./models/{}.model'.format(model_prefix))
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]

import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)